### because of long running tasks and Tokenization of urls (it didnt work locally on Jupyter)
### so please launch it in google collab (it has to work there)

In [122]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [212]:
%matplotlib inline

from tqdm import tqdm
from scipy.sparse import csr_matrix

import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt

import sklearn.model_selection as skl
import sklearn.metrics as sklm
from sklearn.metrics import f1_score

! pip install pymorphy2
! pip install youtokentome
import random
import youtokentome as yttm
import pymorphy2 # Морфологический анализатор

import string # for work with strings
import nltk   # Natural Language Toolkit
from nltk.stem.snowball import SnowballStemmer 

# classifiers
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [213]:
train_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/train.csv")

train_df.head()

,id,url,title,target
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False


In [214]:
test_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/test.csv")

test_df.head()

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


In [215]:
train_df["target"].value_counts()

False    118594
True      16715
Name: target, dtype: int64

In [0]:
X_train_title = train_df["title"].values
X_test_title = test_df["title"].values

X_train_url = train_df["url"].values
X_test_url = test_df["url"].values

y_train = train_df["target"].astype(int).values

## Logistic Regression Classifier

### Url processing

In [217]:
train_data_path = "/content/drive/My Drive/Colab Notebooks/data/train_data.txt"
model_path = "/content/drive/My Drive/Colab Notebooks/data/example.model"

with open(train_data_path, "w") as fout:
    for url in X_train_url:
        print(url, file=fout)

# Training model
yttm.BPE.train(data=train_data_path, vocab_size=4000, model=model_path)

# Loading model
bpe = yttm.BPE(model=model_path)

# Two types of tokenization
print(bpe.encode(["www.pornhub.com"], output_type=yttm.OutputType.SUBWORD))

[['▁www.porn', 'h', 'ub.com']]


#### data cleaning

In [218]:
word_tokenizer = nltk.WordPunctTokenizer()

# get russian and english stop words
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')
stop_words.append(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
def process_data(texts):
    updated_texts = []
    
    # string processers
    stemmer = SnowballStemmer('russian')
    morph = pymorphy2.MorphAnalyzer()
    
    for text in tqdm(texts):    
        text_lower = text.lower() # convert words in a text to lower case
        tokens     = word_tokenizer.tokenize(text_lower) # splits the text into tokens (words)
        
        # remove punct and stop words from tokens
        tokens = [word for word in tokens if (word not in string.punctuation and word not in stop_words)]
        
        tmp_str = ''
        for token in tokens:
            # not taking in account 0 or 1 letter tokens
            if (len(token) >= 2):
                # lemmatization
                token = (morph.parse(token))[0].normal_form
                
                # stemming
                # token = stemmer.stem(token)
                
                tmp_str += token + ' '
        
        updated_texts.append(tmp_str.strip())
    
    return updated_texts

In [220]:
print(X_train_title)
X_train_title_processed = process_data(X_train_title)
X_test_title_processed = process_data(X_test_title)
print(X_train_title_processed[0:10])

  0%|          | 70/135309 [00:00<03:15, 692.23it/s]

['Экс-министр экономики Молдовы - главе МИДЭИ, цель которого сделать из республики не просителя, а донора: Надо избегать долгого нахождения н�'
 'Эта песня стала известна многим телезрителям благодаря сериалу Диверсант-2'
 'Банши 4 сезон 2 серия Бремя красоты смотреть онлайн!' ...
 '1941 Plymouth Special Deluxe Hot Rod, Automatic,Small Block,Power Steering, A/C for sale: photos, technical specifications, description'
 "Купить It's Skin Сыворотка питательная Power 10 Formula YE Effector, 30 мл в Москве - WOWCREAM"
 'Технический спирт в канистрах и флаконах, купить в Москве']


100%|██████████| 165378/165378 [03:43<00:00, 739.91it/s]

['экс министр экономика молдова глава мидэя цель который сделать республика проситель донор избегать долгий нахождение', 'этот песня стать известный многий телезритель благодаря сериал диверсант', 'банша сезон серия бремя красота смотреть онлайн', 'бесить картинка', 'новомосковск сыграть следж хоккеист алексинский звезда сборный китай ...', 'салат корейский морковь копчёный курица кукуруза сухарик калорийность белка жир углевод', 'угловой пневмогайковерт fubag rwc 105 100320 екатеринбург купить цена отзыв характеристика фото инструкция', 'табачный магазин мир табак курительный трубка stanwell курительный труба', 'league of legends page porn comics free online', '08 00 4766р ), одесса ростов дон 20 сентябрь 2019 фаворит чп']


### Splitting and fitting

In [0]:
# url_vectorizer = TfidfVectorizer(max_features = 3500, tokenizer = lambda x: bpe.encode(x, output_type=yttm.OutputType.SUBWORD))
# title_vectorizer = TfidfVectorizer(max_features = 7500)

url_vectorizer = CountVectorizer(max_features = 3500, tokenizer = lambda x: bpe.encode(x, output_type=yttm.OutputType.SUBWORD))
title_vectorizer = CountVectorizer(max_features = 7500)

clf = LogisticRegression(solver='liblinear')

In [0]:
# splitting using function
X_data = []
for i in range(len(X_train_url)):
    X_data.append((X_train_url[i], X_train_title_processed[i]))

df_X_train, df_X_test, df_y_train, df_y_test = skl.train_test_split(X_data, y_train, test_size=0.2, random_state=42)

In [224]:
print('train X:', df_X_train[0:5])
print('test X:', df_X_test[0:5])
print('y_train:', df_y_train[0:20])
print('y_test:', df_y_test[0:20])

train X: [('karat.spb.ru', 'амиксидина продажа цена санкт петербург медицинский дезинфицирующий средство ооо карат "" 367251050'), ('samara.farfor.ru', 'ультрасет new'), ('www.zr.ru', 'lada vesta cross видеообзор презентация серийный машина'), ('xn----htbdmodofzkc0c.xn--p1ai', 'фрезерный станок моделист cnc 6090al 600х900мма продажа цена краснодар фрезерный фрезерный гравировальный станок интернет магазин чпу моделист 49000320'), ('porno365.co', 'популярный порно просмотр порнуха популярный страница 28')]
test X: [('www.kinotrast.com', 'сериал качели 2018 беларусь серия смотреть онлайн бесплатно фильм хороший качество hd 720 1080'), ('pc01.ru', 'шкаф спальный гарнитур энрике 37 000 рубль ., грозный аналог авит'), ('forum.zoneofgames.ru', 'swat close quarters battle русификатор zone of games forum'), ('librebook.me', 'читать онлайн электронный книга прометей жизнь бальзак введение бесплатно регистрация librebook ru'), ('meduniver.com', 'научить щедрость собственный мужчина отучить муж ж

In [0]:
porno_words = ['порно', 'гей', 'лесби', 'хуй', 'член', 'презерватив' ,'секс', 'xxx', 'эротика', 'sex', 'porn', 'dick','cock' ,'gay', 'latina', 'boobs', 'tits']

In [0]:
def get_porn_matrix(data_url, data_title):
    porn_list = []

    for i in range(len(data_url)):
        value = 0
        for pron_word in porno_words:
            if (pron_word in data_url[i] or pron_word in data_title[i]):
            value = 1
            break
        
        porn_list.append(value)

    return csr_matrix(porn_list).transpose()

In [0]:
def get_vectorizers(X_url_train, X_url_test, X_title_train, X_title_test):
    #train
    url_vectorizer.fit(X_url_train)
    url_vectorizer_train = url_vectorizer.transform(X_url_train)

    title_vectorizer.fit(X_title_train)
    title_vectorizer_train = title_vectorizer.transform(X_title_train)

    #test
    url_vectorizer_test = url_vectorizer.transform(X_url_test)
    title_vectorizer_test = title_vectorizer.transform(X_title_test)

    shared_vectorizer_train = scipy.sparse.hstack((url_vectorizer_train, title_vectorizer_train, get_porn_matrix(X_url_train, X_title_train)))
    shared_vectorizer_test = scipy.sparse.hstack((url_vectorizer_test, title_vectorizer_test, get_porn_matrix(X_url_test, X_title_test)))

    return shared_vectorizer_train, shared_vectorizer_test

In [228]:
# fitting vectorizers
df_X_train_urls = [x[0] for x in df_X_train]
df_X_train_titles = [x[1] for x in df_X_train]

df_X_test_urls = [x[0] for x in df_X_test]
df_X_test_titles = [x[1] for x in df_X_test]

shared_vectorizer_train, shared_vectorizer_test = get_vectorizers(df_X_train_urls, df_X_test_urls, df_X_train_titles, df_X_test_titles)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [229]:
%%time
clf.fit(shared_vectorizer_train, df_y_train)

print(clf.n_iter_)

[17]
CPU times: user 1.18 s, sys: 763 ms, total: 1.95 s
Wall time: 1.03 s


In [0]:
y_pred = clf.predict(shared_vectorizer_test)

In [231]:
f1_score(df_y_test, y_pred)
#0.9751107039242632

0.9789680738387049

### Submit Logistic

In [232]:
shared_vectorizer_train, shared_vectorizer_test = get_vectorizers(X_train_url, X_test_url, X_train_title_processed, X_test_title_processed)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [233]:
%%time
clf.fit(shared_vectorizer_train, y_train)

print(clf.n_iter_)

[20]
CPU times: user 1.64 s, sys: 1.06 s, total: 2.7 s
Wall time: 1.43 s


In [0]:
y_pred = clf.predict(shared_vectorizer_test)

In [235]:
test_df["target"] = y_pred.astype(bool)

test_df[["id", "target"]].to_csv("/content/drive/My Drive/Colab Notebooks/data/ml_baseline_logistic.csv", index=False)

!cat ml_baseline.csv | head

cat: ml_baseline.csv: No such file or directory
